# 🔥Causal Graph Neural Networks for Wildfire Danger Prediction🔥
Re-implementation of original work by Zhao et al.(2024) (https://arxiv.org/abs/2403.08414)

IDL S25 Group 23: Wenting Yue, Wenyu Liu, Youyou Huang (Group 23)

## Retrieve files from github repository
If `only notebook` is downloaded locally

In [ ]:
import os
import sys

# Get the current working directory
print("Current working directory:", os.getcwd())
# repo = "https://github.com/youyouh511/11785_IDL_S25_Final-Project.git"
# !git clone {repo}
!git pull

Current working directory: /home/ubuntu/11785_IDL_S25_Final-Project


# Set up

## Environment

Environment setup
```bash
conda env create -f env.yml
```

Activate environment and check device
```bash
conda activate idl_final
python -c "import torch; print('CUDA available:', torch.cuda.is_available())"
nvidia-smi
```

# Imports

In [ ]:
from data import (
    JsonFireDataset
)
from model import (
    AdjacencyMatrix,
    TemporalLSTM,
    CausalGNN
)
# from train import (
    
# )
# from utils import (
# )


import numpy as np
import tqdm
import matplotlib.pyplot as plt
import json
import zipfile
import torch
import requests
import xarray as xr
import yaml
from torchinfo import summary
import shutil
import wandb
import time
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


Using device: cuda


# Config

In [11]:
%%writefile config.yaml

###### Dataset
    root                    : "./data"
    train_json_path         : "train.json"
    val_json_path           : "val.json"
    test_json_path          : "test.json"
    matrix_json_path        : "matrix.json"
    subset                  : 1.0
    batch_size              : 128
    NUM_WORKERS             : 4

    ### Target threshold
    fire_threshold          : 10


###### Model
    ### Adjacency matrix
    local_var_lag           : 8
    oci_var_lag             : 31
    max_lag                 : 312
    independence_test       : "ParCorr"
    tau_max                 : 186
    pc_alpha                : 0.05
    mask_target             : True

    ### Temporal LSTM
    lstm_layer              : 1
    hidden_dim              : 256

    ### GNN
    gnn_nodes               : 7


###### Training
    epochs                  : 30

    lr                      : 1.0e-5
    min_lr                  : 1.0e-9
    
    optimizer               : "Adam"
    betas                   : [0.9, 0.999]
    eps                     : 1.0e-8
    weight_decay            : 5.0e-6

    lr_scheduler            : "CosineAnnealingLR"
    patience                : 10
    early_stop              : True

    save_model              : True
    save_model_path         : "./checkpoints"
    load_model              : False
    load_model_path         : "./checkpoints/best.pth"
    wandb_log               : True
    wandb_project           : "IDL_Final"
    wandb_run_id            : None

Writing config.yaml


In [12]:
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

config

{'root': './data',
 'train_json_path': 'train.json',
 'val_json_path': 'val.json',
 'test_json_path': 'test.json',
 'matrix_json_path': 'matrix.json',
 'subset': 1.0,
 'batch_size': 128,
 'NUM_WORKERS': 4,
 'fire_threshold': 10,
 'local_var_lag': 8,
 'oci_var_lag': 31,
 'max_lag': 312,
 'independence_test': 'ParCorr',
 'tau_max': 186,
 'pc_alpha': 0.05,
 'mask_target': True,
 'lstm_layer': 1,
 'hidden_dim': 256,
 'gnn_nodes': 7,
 'epochs': 30,
 'lr': 1e-05,
 'min_lr': 1e-09,
 'optimizer': 'Adam',
 'betas': [0.9, 0.999],
 'eps': 1e-08,
 'weight_decay': 5e-06,
 'lr_scheduler': 'CosineAnnealingLR',
 'patience': 10,
 'early_stop': True,
 'save_model': True,
 'save_model_path': './checkpoints',
 'load_model': False,
 'load_model_path': './checkpoints/best.pth',
 'wandb_log': True,
 'wandb_project': 'IDL_Final',
 'wandb_run_id': 'None'}

# Data Retrieval & Pre-process

### Download

### Filter

### Sample

# Run data retrieval & filtering

In [35]:
# earth_graph(dataset['FCCI_BA'], 763, 1, latitude2index(latitude), longitude2index(longitude))

# Export json files
# For real implementation, suggest the fire_threshold to be 1?
thredshold = config['fire_threshold']
build_dataset(dataset=dataset, type='train', fire_threshold=thredshold)
build_dataset(dataset=dataset, type='val', fire_threshold=thredshold)
build_dataset(dataset=dataset, type='test', fire_threshold=thredshold)

TypeError: 'NoneType' object is not subscriptable

# Datasets

In [36]:
train_ds = JsonFireDataset(
    json_path   = "train.json",
    local_keys  = ["T2M","TP","VPD_CF"],
    oci_keys    = ["NAO", "NINA34_ANOM"]
)
val_ds = JsonFireDataset(
    json_path   = "val.json",
    local_keys  = ["T2M","TP","VPD_CF"],
    oci_keys    = ["NAO", "NINA34_ANOM"]
)
test_ds = JsonFireDataset(
    json_path   = "test.json",
    local_keys  = ["T2M","TP","VPD_CF"],
    oci_keys    = ["NAO", "NINA34_ANOM"]
)

JSONDecodeError: Expecting ',' delimiter: line 1 column 3739 (char 3738)

# Model

## Adjacency Matrix

In [37]:
matrix_builder = AdjacencyMatrix("train.json", 312)
norm_matrix = matrix_builder.normalize_adj_matrix(matrix_builder.adj_matrix)
mask_matrix = matrix_builder.mask_target(norm_matrix, config['mask_target'])

JSONDecodeError: Expecting ',' delimiter: line 1 column 3739 (char 3738)

## Causal GNN

In [ ]:
model = CausalGNN(
    adj_matrix=norm_matrix,
    num_nodes=config['gnn_nodes'],
    hidden_dim=config['hidden_dim']
).to(device)

model_stats = summary(model)